In [11]:
import sqlite3
import pandas as pd

```sql
CREATE TABLE students (
    student_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    grade TEXT,
    email TEXT,
);
```
```sql
CREATE TABLE courses (
    course_id INTEGER PRIMARY KEY AUTOINCREMENT,
    course_name TEXT,
    credits INTEGER
);
```
```sql
CREATE TABLE enrollments (
    enrollment_id INTEGER PRIMARY KEY AUTOINCREMENT,
    student_id INTEGER,
    course_id INTEGER,
    semester TEXT,
    grade TEXT,
    FOREIGN KEY (student_id) REFERENCES students(student_id),
    FOREIGN KEY (course_id) REFERENCES courses(course_id)
);
```


In [12]:

# Connect to SQLite database (in-memory for demo)
# sql.db
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Create tables
cursor.execute("""
CREATE TABLE students (
    student_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    grade TEXT,
    email TEXT
)
""")
cursor.execute("""
CREATE TABLE courses (
    course_id INTEGER PRIMARY KEY AUTOINCREMENT,
    course_name TEXT,
    credits INTEGER
)
""")
cursor.execute("""
CREATE TABLE enrollments (
    enrollment_id INTEGER PRIMARY KEY AUTOINCREMENT,
    student_id INTEGER,
    course_id INTEGER,
    semester TEXT,
    grade TEXT,
    FOREIGN KEY (student_id) REFERENCES students(student_id),
    FOREIGN KEY (course_id) REFERENCES courses(course_id)
)
""")

# Show all tables in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' and name NOT LIKE 'sqlite_%'")

tables = cursor.fetchall()
print("Tables in database:", [t[0] for t in tables])

Tables in database: ['students', 'courses', 'enrollments']


```sql
INSERT INTO students (name, grade, email) VALUES ('Alice', 'A', 'alice@email.com');
INSERT INTO students (name, grade, email) VALUES ('Bob', 'B', 'bob@email.com');
INSERT INTO courses (course_name, credits) VALUES ('Mathematics', 3);
INSERT INTO courses (course_name, credits) VALUES ('History', 2);
INSERT INTO enrollments (student_id, course_id, semester, grade) VALUES (1, 1, 'Fall2023', 'A');
INSERT INTO enrollments (student_id, course_id, semester, grade) VALUES (2, 2, 'Spring2024', 'B');
```


In [13]:
# Insert data
cursor.execute("INSERT INTO students (name, grade, email) VALUES ('Alice', 'A', 'alice@email.com')")
cursor.execute("INSERT INTO students (name, grade, email) VALUES ('Bob', 'B', 'bob@email.com')")
cursor.execute("INSERT INTO courses (course_name, credits) VALUES ('Mathematics', 3)")
cursor.execute("INSERT INTO courses (course_name, credits) VALUES ('History', 2)")
cursor.execute("INSERT INTO enrollments (student_id, course_id, semester, grade) VALUES (1, 1, 'Fall2023', 'A')")
cursor.execute("INSERT INTO enrollments (student_id, course_id, semester, grade) VALUES (2, 2, 'Spring2024', 'B')")
conn.commit()


# Show all tables after insert
for table_name in [t[0] for t in tables]:
    df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
    print(f"Table: {table_name}")
    display(df)

Table: students


,student_id,name,grade,email
0,1,Alice,A,alice@email.com
1,2,Bob,B,bob@email.com


Table: courses


,course_id,course_name,credits
0,1,Mathematics,3
1,2,History,2


Table: enrollments


,enrollment_id,student_id,course_id,semester,grade
0,1,1,1,Fall2023,A
1,2,2,2,Spring2024,B


```sql

SELECT * FROM students;

SELECT name, grade FROM students WHERE grade = 'A';

SELECT * FROM students WHERE grade <> 'C' ORDER BY name DESC;

SELECT grade, COUNT(*) as num_students FROM students GROUP BY grade;
```


In [14]:

df = pd.read_sql_query("SELECT * FROM students", conn)
display(df)


df = pd.read_sql_query("SELECT name, grade FROM students WHERE grade = 'A'", conn)
display(df)


df = pd.read_sql_query("SELECT * FROM students WHERE grade <> 'C' ORDER BY name DESC", conn)
display(df)


df = pd.read_sql_query("SELECT grade, COUNT(*) as num_students FROM students GROUP BY grade", conn)
display(df)

,student_id,name,grade,email
0,1,Alice,A,alice@email.com
1,2,Bob,B,bob@email.com


,name,grade
0,Alice,A


,student_id,name,grade,email
0,2,Bob,B,bob@email.com
1,1,Alice,A,alice@email.com


,grade,num_students
0,A,1
1,B,1


```sql

UPDATE students SET email = 'alice_new@email.com' WHERE name = 'Alice';

UPDATE courses SET credits = 4 WHERE course_name = 'Mathematics';

UPDATE enrollments SET grade = 'A' WHERE student_id = 2 AND course_id = 2;

UPDATE students SET grade = 'B', email = 'bob_new@email.com' WHERE name = 'Bob';

```


In [15]:

cursor.execute("UPDATE students SET email = 'alice_new@email.com' WHERE name = 'Alice'")

cursor.execute("UPDATE courses SET credits = 4 WHERE course_name = 'Mathematics'")


cursor.execute("UPDATE enrollments SET grade = 'A' WHERE student_id = 2 AND course_id = 2")

cursor.execute("UPDATE students SET grade = 'B', email = 'bob_new@email.com' WHERE name = 'Bob'")
conn.commit()


for table_name in [t[0] for t in tables]:
    df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
    print(f"Table: {table_name}")
    display(df)

Table: students


,student_id,name,grade,email
0,1,Alice,A,alice_new@email.com
1,2,Bob,B,bob_new@email.com


Table: courses


,course_id,course_name,credits
0,1,Mathematics,4
1,2,History,2


Table: enrollments


,enrollment_id,student_id,course_id,semester,grade
0,1,1,1,Fall2023,A
1,2,2,2,Spring2024,A


```sql

SELECT s.name, c.course_name, e.semester, e.grade
FROM students s
JOIN enrollments e ON s.student_id = e.student_id
JOIN courses c ON e.course_id = c.course_id
WHERE e.grade IS NOT NULL
ORDER BY s.name, c.course_name;
```


In [16]:
df = pd.read_sql_query("""
SELECT s.name, c.course_name, e.semester, e.grade
FROM students s
JOIN enrollments e ON s.student_id = e.student_id
JOIN courses c ON e.course_id = c.course_id
WHERE e.grade IS NOT NULL
ORDER BY s.name, c.course_name
""", conn)
display(df)

,name,course_name,semester,grade
0,Alice,Mathematics,Fall2023,A
1,Bob,History,Spring2024,A


```sql
SELECT s.name, COUNT(e.course_id) as courses_taken
FROM students s
LEFT JOIN enrollments e ON s.student_id = e.student_id
GROUP BY s.name
HAVING courses_taken > 0;
```


In [17]:
df = pd.read_sql_query("""
SELECT s.name, COUNT(e.course_id) as courses_taken
FROM students s
LEFT JOIN enrollments e ON s.student_id = e.student_id
GROUP BY s.name
HAVING courses_taken > 0
""", conn)
display(df)

,name,courses_taken
0,Alice,1
1,Bob,1


```sql
SELECT name, (SELECT COUNT(*) FROM enrollments e WHERE e.student_id = s.student_id) as total_enrollments
FROM students s;
```


In [18]:
df = pd.read_sql_query("""
SELECT name, (SELECT COUNT(*) FROM enrollments e WHERE e.student_id = s.student_id) as total_enrollments
FROM students s
""", conn)
display(df)

,name,total_enrollments
0,Alice,1
1,Bob,1


```sql
SELECT name FROM students s
WHERE EXISTS (
    SELECT 1 FROM enrollments e WHERE e.student_id = s.student_id AND e.grade = 'A'
);
```


In [19]:
df = pd.read_sql_query("""
SELECT name FROM students s
WHERE EXISTS (
    SELECT 1 FROM enrollments e WHERE e.student_id = s.student_id AND e.grade = 'A'
)
""", conn)
display(df)

,name
0,Alice
1,Bob


```sql
SELECT s1.name AS student1, s2.name AS student2
FROM students s1
JOIN students s2 ON s1.grade = s2.grade AND s1.student_id <> s2.student_id;
```


In [20]:
df = pd.read_sql_query("""
SELECT s1.name AS student1, s2.name AS student2
FROM students s1
JOIN students s2 ON s1.grade = s2.grade AND s1.student_id <> s2.student_id
""", conn)
display(df)

,student1,student2


```sql
SELECT name FROM students
UNION
SELECT course_name FROM courses;
```


In [21]:
df = pd.read_sql_query("""
SELECT name FROM students
UNION
SELECT course_name FROM courses
""", conn)
display(df)

,name
0,Alice
1,Bob
2,History
3,Mathematics


```sql
SELECT name,
    CASE
        WHEN grade = 'A' THEN 'Excellent'
        WHEN grade = 'B' THEN 'Good'
        ELSE 'Needs Improvement'
    END as performance
FROM students;
```


In [22]:
df = pd.read_sql_query("""
SELECT name,
    CASE
        WHEN grade = 'A' THEN 'Excellent'
        WHEN grade = 'B' THEN 'Good'
        ELSE 'Needs Improvement'
    END as performance
FROM students
""", conn)
display(df)

,name,performance
0,Alice,Excellent
1,Bob,Good


```sql
SELECT
    s.name,
    e.semester,
    e.grade,
    RANK() OVER (PARTITION BY s.student_id ORDER BY e.grade DESC) as grade_rank
FROM students s
JOIN enrollments e ON s.student_id = e.student_id;
```


In [23]:
df = pd.read_sql_query("""
SELECT
    s.name,
    e.semester,
    e.grade,
    RANK() OVER (PARTITION BY s.student_id ORDER BY e.grade DESC) as grade_rank
FROM students s
JOIN enrollments e ON s.student_id = e.student_id
""", conn)
display(df)

,name,semester,grade,grade_rank
0,Alice,Fall2023,A,1
1,Bob,Spring2024,A,1



```sql
DELETE FROM students WHERE name = 'Bob';

DROP TABLE enrollments;
```

In [24]:

cursor.execute("DELETE FROM students WHERE name = 'Bob'")

cursor.execute("DROP TABLE enrollments")
conn.commit()

tables = cursor.execute("SELECT name FROM sqlite_master WHERE type='table' and name NOT LIKE 'sqlite_%'").fetchall()
print("Tables in database:", [t[0] for t in tables])
for table_name in [t[0] for t in tables]:
    df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
    print(f"Table: {table_name}")
    display(df)

Tables in database: ['students', 'courses']
Table: students


,student_id,name,grade,email
0,1,Alice,A,alice_new@email.com


Table: courses


,course_id,course_name,credits
0,1,Mathematics,4
1,2,History,2


In [25]:
# when we're done, we can close the connection
# to our sqlite database

# it runs in memory, so it will be take up ram
# if we don't close it
# and it will not persist after we close the connection
# which means we lose all our data
conn.close()